In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean

import keyword

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math

- 학교와서 들음
- lec11
- 더미변수 추가 부터


# 더미변수

- 범주의 갯수 만큼 더미변수를 만들어보자
- 어케 파이썬에서 만드냐


In [6]:
df=pd.read_csv('c:/temp/education2020.csv')
df

,Unnamed: 0,year,metro,id,sex,age,number,education,marriage,asset,debt,income,income_d,industry,job,house,education_year
0,0,2020,G1,10000112,1,34,3,6,2,112000,54500,6593,4599,F,3,2,16
1,1,2020,G1,10000132,2,45,2,8,2,42500,17500,17720,15257,J,2,3,21
2,2,2020,G1,10000162,2,73,1,2,3,5712,0,908,725,T,4,2,6
3,3,2020,G1,10000182,1,58,2,4,2,14870,0,2748,2431,C,5,2,12
4,4,2020,G1,10000192,2,27,1,4,1,814,0,1015,893,R,2,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18059,18059,2020,G2,993800291,1,42,3,6,2,6705,2200,179,-390,NaN,NaN,3,16
18060,18060,2020,G2,994800251,1,49,5,6,2,130180,22000,15454,11784,O,3,1,16
18061,18061,2020,G2,994800261,1,57,5,6,2,265226,29000,15098,11560,NaN,NaN,1,16
18062,18062,2020,G2,994800291,2,53,2,7,4,109695,25000,9114,7074,O,2,2,18


In [9]:
sex_dummies=pd.get_dummies(df['sex'])
sex_dummies

,1,2
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
18059,1,0
18060,1,0
18061,1,0
18062,0,1


In [10]:
job_dummies=pd.get_dummies(df['job'],prefix='j')
job_dummies

,j_1,j_2,j_3,j_4,j_5,j_6,j_7,j_8,j_9,j_A
0,0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
18059,0,0,0,0,0,0,0,0,0,0
18060,0,0,1,0,0,0,0,0,0,0
18061,0,0,0,0,0,0,0,0,0,0
18062,0,1,0,0,0,0,0,0,0,0


In [11]:
# 9개의 더미변수 생성됨
#회기분석에서 범주형 변수는 먼저 더미변수로 만들자

In [12]:
df2=df[['income','age','education_year']]
df3=pd.concat([df2,sex_dummies,job_dummies],axis=1)
#axis=1 : 칼럼을 추가하는 방향으로 (가로 방햐으로) 같다 붙여라

In [13]:
df3

,income,age,education_year,1,2,j_1,j_2,j_3,j_4,j_5,j_6,j_7,j_8,j_9,j_A
0,6593,34,16,1,0,0,0,1,0,0,0,0,0,0,0
1,17720,45,21,0,1,0,1,0,0,0,0,0,0,0,0
2,908,73,6,0,1,0,0,0,1,0,0,0,0,0,0
3,2748,58,12,1,0,0,0,0,0,1,0,0,0,0,0
4,1015,27,12,0,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18059,179,42,16,1,0,0,0,0,0,0,0,0,0,0,0
18060,15454,49,16,1,0,0,0,1,0,0,0,0,0,0,0
18061,15098,57,16,1,0,0,0,0,0,0,0,0,0,0,0
18062,9114,53,18,0,1,0,1,0,0,0,0,0,0,0,0


In [14]:
edu_model12=ols('income~age+education_year',data=df3).fit()
print(edu_model12)

NameError: name 'ols' is not defined

In [15]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [16]:
from statsmodels.formula.api import ols

In [18]:
edu_model12=ols('income~age+education_year',data=df3).fit()
print(edu_model12.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     1773.
Date:                Mon, 27 May 2024   Prob (F-statistic):               0.00
Time:                        12:22:00   Log-Likelihood:            -1.7980e+05
No. Observations:               18064   AIC:                         3.596e+05
Df Residuals:                   18061   BIC:                         3.596e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        322.1142    277.944      1.

In [19]:
# 아래에는 범주 더미변수 포함해서 ols 해보자
# 더미변수는 서로 반대임 ex. 남자/여자, 그러니까 두개 더미로 만들어도, 하나만 ㅡㅆ면 됨 -> dummy variable trap, 회귀분석 수업에서 알아보시길...

In [22]:
sex_dummies=pd.get_dummies(df['sex'],prefix='s')
sex_dummies


,s_1,s_2
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
18059,1,0
18060,1,0
18061,1,0
18062,0,1


In [23]:
edu_model3=ols('income~age+education_year+s_1+j_2+j_3+j_4+j_5+j_6+j_7+j_8+j_9',data=df3).fit()
print(edu_model12.summary())

PatsyError: Error evaluating factor: NameError: name 's_1' is not defined
    income~age+education_year+s_1+j_2+j_3+j_4+j_5+j_6+j_7+j_8+j_9
                              ^^^

In [25]:
# 이 위에 하면서 먼 설명있었는데 써야 함......ㅠ.ㅠㅠ.ㅠ
# 여기까지가 lec11
# 유의성 검저엥 해당하는 std err, t, p , ㄷ ㅏ중요함. coef만큼 유의성 검정에 대한 값들도 중요함

# lec 12 회귀분석과 머신러닝(1)

- 하... 다시 듣자 당연히..
- 경사: 기울기
- w: weight 가중치 (기울기) , b: bias 편의 (상수항)
- 경사하강법을 통해 w와 b를 찾고자 하는 것

## 경사하강법
- 어떤 점에서 접선의 기울기를 구해봐, 그게 플러스면 왼쪽으로 가게 함. 그래야 극점에 도달.
- 마이너스면 오른쪽으로 가게함, 그러다 보면 극점에 도달하게됨
- 목적함수를 최소화하는 w와 b를 찾는 것
- 중간을 못들음 ㅎㅎ
- ㅇㅇ...ㅎㅎ..몰러
- 비용함수가 볼록함수면, 초기값을 어디에서 정의 하든지 경사하강법 이용하면 비용함수가 최소 되ㅡㄴ 가주잋 w 구할 수 이싿
- 볼록함수란?: 필기하기
- w 업데이트 시키기: parameter update
- alpha: learning rate 학습률, 파이썬에서는 보통 되게 작은 값으로 직접 지정해준다.
- 업데이트 할 때 경사의 일부분을 빼준다. 얼마나 빼줄지 결정하는 게 알파.
- 알파는 미리 정해줌
- 알파를 미리 정할 때 , 그 값이 작거나 크면 어케되는 거야?? -> 다음 시간에